## Linting System Build-Up

This notebook demonstrates using the built-in tool providers to run Black, Ruff, Mypy, and Radon.

**Assumptions**:
- `black`, `ruff`, and `mypy` are installed.
- `radon` may be missing; errors are captured to show failure handling.

Logging is configured at the DEBUG level so provider actions are visible.

In [1]:
import logging
from importlib import import_module
from pathlib import Path
import subprocess
import black
import ruff
import mypy
import radon

from app.factories.tool_provider import ToolProviderFactory

import_module("app.extensions.tool_providers")
logging.basicConfig(level=logging.DEBUG)


Define code snippets used by the linters. We include valid code along with intentionally bad examples to trigger linting and typing errors.

In [2]:
good_code = "def add(x: int, y: int) -> int:
    return x + y
"
bad_linting = "x=1


def foo():
    return 1
"
bad_typing = "def add(x: int, y: int) -> int:
    return str(x + y)
"
complex_code = (
    "def complex(n):\n"
    "    for i in range(n):\n"
    "        if i % 2 == 0:\n"
    "            for j in range(n):\n"
    "                if j % 3 == 0:\n"
    "                    print(i, j)\n"
)
missing_import = "import os


def bar():
    return 42
"
invalid_code = "def oops(:
    pass
"


### Running Black

In [3]:
def run_black(code):
    path = Path("black_demo.py")
    path.write_text(code)
    provider = ToolProviderFactory.create("black")
    try:
        result = provider.run(str(path))
        print(result.stdout)
        print(result.stderr)
        print(path.read_text())
    finally:
        path.unlink()

print('Black on good code:')
run_black(good_code)
print('Black on linting issues:')
run_black(bad_linting)


### Running Ruff

In [4]:
def run_ruff(code):
    path = Path("ruff_demo.py")
    path.write_text(code)
    provider = ToolProviderFactory.create("ruff")
    try:
        result = provider.run(str(path))
        print(result.stdout)
        print(result.stderr)
        print('return code', result.returncode)
    finally:
        path.unlink()

print('Ruff on good code:')
run_ruff(good_code)
print('Ruff with unused import:')
run_ruff(missing_import)
print('Ruff on invalid code:')
run_ruff(invalid_code)


### Running Mypy

In [5]:
def run_mypy(code):
    path = Path("mypy_demo.py")
    path.write_text(code)
    provider = ToolProviderFactory.create("mypy")
    try:
        result = provider.run(str(path))
        print(result.stdout)
        print(result.stderr)
        print('return code', result.returncode)
    finally:
        path.unlink()

print('Mypy on good code:')
run_mypy(good_code)
print('Mypy with typing error:')
run_mypy(bad_typing)
print('Mypy on invalid code:')
run_mypy(invalid_code)


### Running Radon

In [6]:
def run_radon(code):
    path = Path("radon_demo.py")
    path.write_text(code)
    provider = ToolProviderFactory.create("radon")
    try:
        result = provider.run(str(path))
        print(result.stdout)
    except Exception as e:
        print('Radon failed:', e)
    finally:
        path.unlink()

print('Radon on good code:')
run_radon(good_code)
print('Radon on complex code:')
run_radon(complex_code)
